In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
file_path = './assets/diabetes.csv'
df = pd.read_csv(file_path)
df.replace('?', pd.NA, inplace=True)
df.dropna(inplace=True)

In [4]:
correlation_matrix = df.corr()
correlation_matrix

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [165]:
drop_columns = ['BloodPressure', 'SkinThickness', 'Insulin']

df = df.drop(columns=drop_columns)

In [10]:
X = df.drop(columns=['Outcome'])
y = df['Outcome']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
model = Sequential()

model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
from sklearn.feature_selection import SelectKBest, chi2

k = 5

selector = SelectKBest(score_func=chi2, k=k)

X_train_selected = selector.fit_transform(X_train, y_train)

model = Sequential()

model.add(Dense(16, input_dim=X_train_selected.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, epochs=100, batch_size=60, verbose=2)
# model.fit(X_train, y_train, epochs=100)
y_pred = model.predict(X_test)

y_pred_binary = np.where(y_pred > 0.5, 1, 0)

accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy on test set: {accuracy}')

Epoch 1/100


ValueError: in user code:

    File "C:\Users\barar\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\barar\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\barar\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\barar\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\barar\AppData\Roaming\Python\Python39\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\barar\AppData\Roaming\Python\Python39\site-p

In [18]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

k = 5

selector = SelectKBest(score_func=chi2, k=k)

X_train_selected = selector.fit_transform(X_train, y_train)

model = Sequential()

model.add(Dense(16, input_dim=X_train_selected.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_selected, y_train, epochs=100, batch_size=50, verbose=2)

X_test_selected = selector.transform(X_test)

y_pred = model.predict(X_test_selected)

y_pred_binary = np.where(y_pred > 0.5, 1, 0)

accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy on test set: {accuracy}')


Epoch 1/100
11/11 - 1s - loss: 4.9061 - accuracy: 0.4190 - 839ms/epoch - 76ms/step
Epoch 2/100
11/11 - 0s - loss: 3.0250 - accuracy: 0.5493 - 19ms/epoch - 2ms/step
Epoch 3/100
11/11 - 0s - loss: 1.9207 - accuracy: 0.5363 - 24ms/epoch - 2ms/step
Epoch 4/100
11/11 - 0s - loss: 1.6943 - accuracy: 0.5326 - 20ms/epoch - 2ms/step
Epoch 5/100
11/11 - 0s - loss: 1.4284 - accuracy: 0.5289 - 21ms/epoch - 2ms/step
Epoch 6/100
11/11 - 0s - loss: 1.2640 - accuracy: 0.4860 - 23ms/epoch - 2ms/step
Epoch 7/100
11/11 - 0s - loss: 1.1405 - accuracy: 0.5102 - 27ms/epoch - 2ms/step
Epoch 8/100
11/11 - 0s - loss: 1.0811 - accuracy: 0.4935 - 28ms/epoch - 3ms/step
Epoch 9/100
11/11 - 0s - loss: 1.0449 - accuracy: 0.4935 - 26ms/epoch - 2ms/step
Epoch 10/100
11/11 - 0s - loss: 1.0137 - accuracy: 0.5158 - 28ms/epoch - 3ms/step
Epoch 11/100
11/11 - 0s - loss: 0.9429 - accuracy: 0.5214 - 25ms/epoch - 2ms/step
Epoch 12/100
11/11 - 0s - loss: 0.9124 - accuracy: 0.5400 - 26ms/epoch - 2ms/step
Epoch 13/100
11/11 - 0s